In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [2]:
api_key = ""
model = "gpt-4.1"
llm = ChatOpenAI(model=model, api_key=api_key)

In [3]:
PROMPT = {
    "system": """당신은 금융 전문가이자 퀀트 애널리스트입니다.  
아래에 제시하는 "금융 가설"을 아래의 규칙을 정확히 지켜서 명확한 "IF-THEN-ELSE" 형태로 변환해주세요.

- **규칙**:
  - 표현 형태는 항상 "IF [조건들] THEN [결과1] ELSE [결과2]" 형식을 따릅니다.
  - 조건이 여러 개일 경우 "AND" 또는 "OR"로 연결해 주세요.
  - 수치나 파라미터가 필요한 경우 반드시 명시해 주세요.
  - 모든 표현은 최대한 간결하고 명확하게 작성해주세요.
  - 자연어 설명은 넣지 말고 오직 IF-THEN-ELSE 표현만 생성해 주세요.
  - 사용하는 함수의 예시는 다음과 같습니다:
    - 가격 데이터: PRICE("close"), PRICE("high"), PRICE("low"), PRICE("open")
    - 거래량 데이터: VOLUME()
    - 통계값 함수: MAX(data, 기간), MIN(data, 기간), AVG(data, 기간), SUM(data, 기간)
    - 기술적 지표: RSI(기간), SMA(data, 기간), EMA(data, 기간)

아래에 실제 예시도 드리겠습니다.

- **금융 가설 예시**:
"최근 10일간의 최고가를 오늘 종가가 돌파하고 거래량이 최근 10일 평균 거래량보다 20% 이상 높다면, 상승 신호로 간주한다."

- **IF-THEN-ELSE 표현 예시**:
IF PRICE("close") > MAX(PRICE("high"), 10) AND VOLUME() > AVG(VOLUME(), 10)*1.2 THEN 1 ELSE 0

위 규칙과 예시를 참고하여, 다음 금융 가설을 IF-THEN-ELSE 표현으로 변환해주세요.
""",
    "human": """hypothesis: {hypothesis}""",
}

prompt_ = ChatPromptTemplate.from_messages(
    [
        ("system", PROMPT["system"]),
        ("human", PROMPT["human"]),
    ]
)

chain = prompt_ | llm

answer = chain.invoke({"hypothesis": "최근 종가가 20일 이동평균선을 상향 돌파(골든크로스)하고, 거래량이 최근 10일 평균보다 많다면, 주가가 상승 추세로 진입할 가능성이 크다"})

In [4]:
print(answer.content)

IF PRICE("close") > SMA(PRICE("close"), 20) AND PRICE("close", 1) <= SMA(PRICE("close"), 20, 1) AND VOLUME() > AVG(VOLUME(), 10) THEN 1 ELSE 0


In [5]:
from langchain_experimental.tools import PythonREPLTool

# 파이썬 코드를 실행하는 도구를 생성합니다.
python_tool = PythonREPLTool()


In [6]:
from langchain.agents import initialize_agent, AgentType

In [12]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 금융 데이터 분석 및 자동매매 시스템 구현에 특화된 전문적인 Python 프로그래머입니다."),
    ("human", """
다음 AST와 종목 티커를 기반으로 알파 신호를 계산하는 Python 코드를 작성하고 실행해주세요.

AST (투자 전략):
{ast}

종목 티커 (Ticker):
{ticker}

다음 조건을 엄격히 지켜 코드를 작성하세요:


규칙 구현 요구:
1. yfinance로 최근 60일 일봉 다운로드
2. 동일 섹터 평균은 SPDR ETF 매핑 등 간단히 ‘XLK’ 같은 섹터 ETF 사용
3. 정수 0 또는 1을 print (외부 텍스트 금지)
""")
])

ast = "IF RSI(14) < 50 THEN 1 ELSE 0"
ticker = "NKE"

formatted_prompt = prompt_template.format_messages(ast=ast, ticker=ticker)

agent = initialize_agent(
    [python_tool],  # 사용할 도구 리스트
    llm,  # 사용할 언어 모델
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # 에이전트 타입
    verbose=True  # 상세 출력 설정
)

result = agent.run(formatted_prompt)



> Entering new AgentExecutor chain...
Thought: 전략은 'RSI(14) < 50면 1, 아니면 0'입니다. NKE의 60일 데이터를 yfinance로 받고, RSI(14)를 계산해야 합니다. 결과 조건에 따라 0 또는 1을 출력합니다.
Action: Python_REPL
Action Input: 
import yfinance as yf
import pandas as pd

def rsi(series: pd.Series, window: int = 14) -> pd.Series:
    delta = series.diff()
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)
    avg_gain = gain.rolling(window, min_periods=window).mean()
    avg_loss = loss.rolling(window, min_periods=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

ticker = 'NKE'
data = yf.download(ticker, period='60d', interval='1d')
close = data['Close']
rsi_14 = rsi(close).iloc[-1]

alpha = 1 if rsi_14 < 50 else 0
print(alpha)

[*********************100%***********************]  1 of 1 completed



Observation: ValueError('The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().')
Thought:

[*********************100%***********************]  1 of 1 completed

Thought: RSI 계산 함수가 마지막 값을 반환하는 것이 아니라 전체 시리즈를 반환했습니다. iloc[-1]을 사용해 마지막 날 RSI만 추출해야 하며, NaN 체크도 필요합니다. 다시 정확하게 코드를 작성해서 실행하겠습니다.
Action: Python_REPL
Action Input: 
import yfinance as yf
import pandas as pd

def rsi(series: pd.Series, window: int = 14) -> pd.Series:
    delta = series.diff()
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)
    avg_gain = gain.rolling(window, min_periods=window).mean()
    avg_loss = loss.rolling(window, min_periods=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

ticker = "NKE"
data = yf.download(ticker, period="60d", interval="1d")
close = data["Close"]
rsi_series = rsi(close)
last_rsi = rsi_series.iloc[-1]
alpha = 1 if last_rsi < 50 else 0
print(int(alpha))
Observation: ValueError('The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().')
Thought:


[*********************100%***********************]  1 of 1 completed

Thought: 오류 메시지로 보아 last_rsi가 스칼라 값이 아니라 Series나 NaN일 가능성이 있습니다. rolling 적용 후 NaN이 생길 수 있으므로, iloc[-1] 값이 NaN인지 먼저 체크하여 처리해야 합니다. 만약 NaN이면 0, 아니면 비교를 수행합니다.
Action: Python_REPL
Action Input: 
import yfinance as yf
import pandas as pd
import numpy as np

def rsi(series: pd.Series, window: int = 14) -> pd.Series:
    delta = series.diff()
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)
    avg_gain = gain.rolling(window, min_periods=window).mean()
    avg_loss = loss.rolling(window, min_periods=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

ticker = "NKE"
data = yf.download(ticker, period="60d", interval="1d")
close = data["Close"]
rsi_series = rsi(close)
last_rsi = rsi_series.iloc[-1]

if np.isnan(last_rsi):
    print(0)
else:
    print(1 if last_rsi < 50 else 0)
Observation: ValueError('The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().')
Thought:

Thought: ValueError('The truth value of a Series is ambiguous.') 메시지는 last_rsi가 여전히 단일 스칼라값이 아니라 시리즈 또는 또 다른 형태라는 의미입니다. iloc[-1]이 NaN이면 float('nan')이고, 아니면 단일 float이어야 합니다. 혹시 데이터프레임 인덱스 문제일 수 있으므로, last_rsi 값과 형을 프린트해 확인해보겠습니다.
Action: Python_REPL
Action Input: 
print(type(rsi_series.iloc[-1]))
print(rsi_series.iloc[-1])
Observation: <class 'pandas.core.series.Series'>
Ticker
NKE    40.445031
Name: 2025-04-30 00:00:00, dtype: float64

Thought:Thought: rsi 함수가 pandas.Series를 반환하고, yfinance의 DataFrame에서 'Close' 컬럼을 사용해 rsi(close) 시리즈가 생성됩니다. 하지만 yfinance가 여러 티커 입력을 처리할 때 결과가 MultiIndex DataFrame이 되어 Series가 아닌 구조를 반환할 수 있으며, rsi_series의 각 요소도 Series가 되는 문제가 생긴 것입니다. 실제로 rsi_series.iloc[-1]이 Series임을 확인했습니다. 단일 ticker(NKE)에 대해 결과가 float이 되도록 명확하게 인덱싱을 보장해야 합니다. 만약 rsi_series가 Series of Series라면, 마지막 값의 첫 번째 값만 사용하면 됩니다.

Action: Python_REPL
Action Input: 
# rsi_series.iloc[-1]의 내용을 명확하게 float로 추출
if isinstance(rsi_series.iloc[-1], pd.Series):
    last_rsi = rsi_series.iloc[-1].values[0]

In [13]:
print(result)

1
